In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tqdm import tqdm
import pandas as pd
from collections import OrderedDict
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
torch.cuda.empty_cache

## Declaring some Functions and Variables

In [ ]:
def Graph(Epochs,accuracy,loss):
    plt.figure(figsize=(10,5))
    plt.plot(range(Epochs), accuracy, color='green')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()    


    plt.figure(figsize=(10,5))
    plt.plot(range(Epochs), loss, color='red')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show() 


In [ ]:
def Accuracy_Scores(outputs,binary_labels,correct,total):

    scores = precision_recall_fscore_support(binary_labels, outputs, average='weighted')
    print(' Precision:', scores[0])
    print(' Recall:', scores[1])
    print(' F1 Score:', scores[2])  
    print(' Accuracy: %d %%' % (100 * correct / total)) 

In [ ]:
A_labels = ["Actual False", "Actual True"]
P_labels = ["Predicted False", "Predicted True"]   
Epochs = 7

In [ ]:
data_dir = 'A_05_Part_02_Dataset'

## Loading Dataset

In [ ]:
class ImagePath(datasets.ImageFolder):
    
    #overriding
    def __getitem__(self, index):
        original_tuple = super(ImagePath, self).__getitem__(index)
        path = self.imgs[index][0]
        return (original_tuple + (path,))

In [ ]:

# Defining transformations
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])


# Applying transformations on the data
train_data = ImagePath(data_dir + '/Train', transform=train_transforms)
valid_data = ImagePath(data_dir + '/Validation', transform=valid_transforms)
test_data = ImagePath(data_dir + '/Test', transform=test_transforms)


# Data Loaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=70, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=60, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=60, shuffle=True)


print("**** Classes ****")
class_names = tuple(train_data.classes)
print(class_names)

In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# Get a batch of training data
inputs, classes,paths = next(iter(trainloader))
show_databatch(inputs, classes)

# BCE LOGISTLOSS


## VGG16

### Loading pre-trained


In [ ]:
vgg16 = models.vgg16(pretrained=True)


### Freezing the layers


In [ ]:
for param in vgg16.features.parameters():
    param.require_grad = False
    

### Modificatiion in Layers

In [ ]:
# Removing last layer of VGG-16
num_features = vgg16.classifier[6].in_features
classifier = list(vgg16.classifier.children())[:-1]
classifier.extend([nn.Linear(num_features, len(class_names))])
vgg16.classifier = nn.Sequential(*classifier)

print(vgg16)

 ### Function & Optimizer

In [ ]:
criterion = nn.BCEWithLogitsLoss()
vgg16_optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

### Training

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

vgg16.train()

vgg16_accuracy = []

vgg16_loss = []

# Running Epochs
for epoch in range(Epochs):

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels, paths = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        vgg16_optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vgg16(inputs)             #----> forward pass
        
        # One-Hot encoding labels
        encoder = OneHotEncoder(handle_unknown='ignore')
        labels_in_numpy = torch.Tensor.cpu(labels).detach().numpy().reshape(-1, 1)
        categories = np.array([[0], [1], [2]])
        encoder.fit(categories)
        binary_labels = encoder.transform(labels_in_numpy).toarray()
           
        loss = criterion(outputs, torch.from_numpy(binary_labels).to(device))   #----> compute loss
        loss.backward()                     #----> backward pass
        vgg16_optimizer.step()              #----> weights update
        
        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))
    
    vgg16_loss.append(loss.data)
        
    torch.save(vgg16.state_dict(), 'vgg16_withoutfocal.pth')
    correct = 0
    total = 0

    covid_Matrix = np.zeros((2,2))
    pneu_Matrix = np.zeros((2,2))
    normal_Matrix = np.zeros((2,2))
    
    with torch.no_grad():
        for data in validloader:
            images, labels, paths = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            
            # Applying sigmoid
            outputs = 1 / (1 + np.exp(-torch.Tensor.cpu(outputs).detach().numpy()))
            
            # Converting to 1 if greater than threshold of 0.5
            # if no value is greater than the threshold then defaulting to convert the max value to 1
            outputs[:] = np.where(outputs == outputs.max(axis=1).reshape(-1, 1), 1, outputs)
            outputs = np.where(outputs >= 0.5, 1, 0)
            
            # Converting output back to tensor 
            outputs = torch.from_numpy(outputs).to(device)
            
            # One-Hot encoding labels
            encoder = OneHotEncoder(handle_unknown='ignore')
            labels_in_numpy = torch.Tensor.cpu(labels).detach().numpy().reshape(-1, 1)
            categories = np.array([[0], [1], [2]])
            encoder.fit(categories)
            binary_labels = encoder.transform(labels_in_numpy).toarray()
            
            # Setting pneumonia to 1 if covid-19 is seen
            for row in range(binary_labels.shape[0]):
                if binary_labels[row][0] == 1:
                    binary_labels[row][2] = 1
                    
            binary_labels = torch.from_numpy(binary_labels).to(device)
               
            # Converting to numpy arrays for comparison
            outputs = torch.Tensor.cpu(outputs).detach().numpy()
            binary_labels = torch.Tensor.cpu(binary_labels).detach().numpy()
            
            
            total += labels.size(0)
            correct += np.all(outputs==binary_labels, axis=1).sum()
            
            # Computing confusion matrices
            mat = multilabel_confusion_matrix(binary_labels, outputs)
            
            ## Making Confusion Matrix

            normal_Matrix[0][0] += mat[2][0][0]
            normal_Matrix[0][1] += mat[2][0][1]
            normal_Matrix[1][0] += mat[2][1][0]
            normal_Matrix[1][1] += mat[2][1][1]

            covid_Matrix[0][0]  += mat[0][0][0]
            covid_Matrix[0][1]  += mat[0][0][1]
            covid_Matrix[1][0] += mat[0][1][0]
            covid_Matrix[1][1] += mat[0][1][1]
            
            pneu_Matrix[0][0]  += mat[1][0][0]
            pneu_Matrix[0][1] += mat[1][0][1]
            pneu_Matrix[1][0] += mat[1][1][0]
            pneu_Matrix[1][1] += mat[1][1][1]
            

    
    
    vgg16_accuracy.append((100 * correct / total))

### Scores and Confusion

In [ ]:
print('********* VGG16 Confusion Matrices ***********')

Accuracy_Scores(outputs,binary_labels,correct,total)
   
print('\n\n Normal ')
print(pd.DataFrame(normal_Matrix, A_labels, P_labels))
print('\n\n Covid-19 ')
print(pd.DataFrame(covid_Matrix, A_labels, P_labels))
print('\n\n Pneumonia ')
print(pd.DataFrame(pneu_Matrix, A_labels, P_labels))



### Accuracy and Loss Curves

In [ ]:
print('********* VGG16 ***********')

Graph(Epochs,vgg16_accuracy,vgg16_loss)


### Testing

In [ ]:
# Decalring a Dataframe
testing_df = pd.DataFrame()

with torch.no_grad():
    for data in testloader:
        images, labels, paths = data
        images, labels = images.to(device), labels.to(device)
        
        outputs = vgg16(images)
        
        # Applying sigmoid
        outputs = 1 / (1 + np.exp(-torch.Tensor.cpu(outputs).detach().numpy()))
            
        # Converting to 1 if greater than threshold of 0.5
        # if no value is greater than the threshold then defaulting to convert the max value to 1
        outputs[:] = np.where(outputs == outputs.max(axis=1).reshape(-1, 1), 1, outputs)
        outputs = np.where(outputs >= 0.5, 1, 0)
      
        Imagepaths = pd.DataFrame(paths)[0].str.split("/", expand = True).iloc[:,-1]
        batch_df = pd.concat([Imagepaths, pd.DataFrame(outputs)], axis=1, join='inner')
        columns_titles = [9, 0, 2, 1]
        batch_df = batch_df.reindex(columns=columns_titles)
        batch_df.columns = [''] * len(batch_df.columns)
        batch_df.index = [''] * len(batch_df.index)
        
        testing_df = testing_df.append(batch_df)

testing_df.reset_index(drop=True)

testing_df.to_csv('MSDS19033_results1.csv', index=False, header=False)


## ResNet-18

### Loading pre-trained


In [ ]:
resnet18 = models.resnet18(pretrained=True)

### Freezing Layers

In [ ]:
for param in resnet18.parameters():
    param.require_grad = False

### Modification in Layers

In [ ]:
resnet18.fc = nn.Sequential(OrderedDict([('fc1', nn.Linear(512, len(class_names)))]))
print(resnet18)

 ### Function & Optimizer

In [ ]:

criterion = nn.BCEWithLogitsLoss()
resnet18_optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)

### Training

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet18.to(device)

resnet18.train()

resnet18_accuracy = []

resnet18_loss = []

# Running Epochs
for epoch in range(Epochs):

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels, paths = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        resnet18_optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet18(inputs)             #----> forward pass
        
        # One-Hot encoding labels
        encoder = OneHotEncoder(handle_unknown='ignore')
        labels_in_numpy = torch.Tensor.cpu(labels).detach().numpy().reshape(-1, 1)
        categories = np.array([[0], [1], [2]])
        encoder.fit(categories)
        binary_labels = encoder.transform(labels_in_numpy).toarray()
           
        loss = criterion(outputs, torch.from_numpy(binary_labels).to(device))   #----> compute loss
        loss.backward()                     #----> backward pass
        resnet18_optimizer.step()              #----> weights update
        
        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))
    
    resnet18_loss.append(loss.data)
        
    torch.save(resnet18.state_dict(), 'resnet18_withoutfocal.pth')
    
    correct = 0
    total = 0

    covid_Matrix = np.zeros((2,2))
    pneu_Matrix = np.zeros((2,2))
    normal_Matrix = np.zeros((2,2))
    
    with torch.no_grad():
        for data in validloader:
            images, labels, paths = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            
            # Applying sigmoid
            outputs = 1 / (1 + np.exp(-torch.Tensor.cpu(outputs).detach().numpy()))
            
            # Converting to 1 if greater than threshold of 0.5
            # if no value is greater than the threshold then defaulting to convert the max value to 1
            outputs[:] = np.where(outputs == outputs.max(axis=1).reshape(-1, 1), 1, outputs)
            outputs = np.where(outputs >= 0.5, 1, 0)
            
            # Converting output back to tensor 
            outputs = torch.from_numpy(outputs).to(device)
            
            # One-Hot encoding labels
            encoder = OneHotEncoder(handle_unknown='ignore')
            labels_in_numpy = torch.Tensor.cpu(labels).detach().numpy().reshape(-1, 1)
            categories = np.array([[0], [1], [2]])
            encoder.fit(categories)
            binary_labels = encoder.transform(labels_in_numpy).toarray()
            
            # Setting pneumonia to 1 if covid-19 is seen
            for row in range(binary_labels.shape[0]):
                if binary_labels[row][0] == 1:
                    binary_labels[row][2] = 1
                    
            binary_labels = torch.from_numpy(binary_labels).to(device)
               
            # Converting to numpy arrays for comparison
            outputs = torch.Tensor.cpu(outputs).detach().numpy()
            binary_labels = torch.Tensor.cpu(binary_labels).detach().numpy()
            
            
            total += labels.size(0)
            correct += np.all(outputs==binary_labels, axis=1).sum()
            
            # Computing confusion matrices
            mat = multilabel_confusion_matrix(binary_labels, outputs)
            
            ## Making Confusion Matrix

            normal_Matrix[0][0] += mat[2][0][0]
            normal_Matrix[0][1] += mat[2][0][1]
            normal_Matrix[1][0] += mat[2][1][0]
            normal_Matrix[1][1] += mat[2][1][1]


            covid_Matrix[0][0]  += mat[0][0][0]
            covid_Matrix[0][1]  += mat[0][0][1]
            covid_Matrix[1][0] += mat[0][1][0]
            covid_Matrix[1][1] += mat[0][1][1]
            
            pneu_Matrix[0][0]  += mat[1][0][0]
            pneu_Matrix[0][1] += mat[1][0][1]
            pneu_Matrix[1][0] += mat[1][1][0]
            pneu_Matrix[1][1] += mat[1][1][1]
            

    
    
    resnet18_accuracy.append((100 * correct / total))

### Scores and Confusion Matrix

In [ ]:
print('********* ResNet18 Confusion Matrices *********')
Accuracy_Scores(outputs,binary_labels,correct,total)


print('\n\n Normal ')
print(pd.DataFrame(normal_Matrix, A_labels, P_labels))
print('\n\n Covid-19 ')
print(pd.DataFrame(covid_Matrix, A_labels, P_labels))
print('\n\n Pneumonia ')
print(pd.DataFrame(pneu_Matrix, A_labels, P_labels))



### Accuracy and Loss Curve

In [ ]:
print('********* ResNet18 *********')

Graph(Epochs,resnet18_accuracy,resnet18_loss)


### Testing

In [ ]:
# Decalring a Dataframe
testing_df = pd.DataFrame()

with torch.no_grad():
    for data in testloader:
        images, labels, paths = data
        images, labels = images.to(device), labels.to(device)
        
        outputs = resnet18(images)
        
        # Applying sigmoid
        outputs = 1 / (1 + np.exp(-torch.Tensor.cpu(outputs).detach().numpy()))
            
        # Converting to 1 if greater than threshold of 0.5
        # if no value is greater than the threshold then defaulting to convert the max value to 1
        outputs[:] = np.where(outputs == outputs.max(axis=1).reshape(-1, 1), 1, outputs)
        outputs = np.where(outputs >= 0.5, 1, 0)
      
        Imagepaths = pd.DataFrame(paths)[0].str.split("/", expand = True).iloc[:,-1]
        batch_df = pd.concat([Imagepaths, pd.DataFrame(outputs)], axis=1, join='inner')
        columns_titles = [9, 0, 2, 1]
        batch_df = batch_df.reindex(columns=columns_titles)
        batch_df.columns = [''] * len(batch_df.columns)
        batch_df.index = [''] * len(batch_df.index)
        
        testing_df = testing_df.append(batch_df)

testing_df.reset_index(drop=True)

testing_df.to_csv('MSDS19033_results2.csv', index=False, header=False)


# Focal Loss

## VGG-16

In [ ]:
vgg16 = models.vgg16(pretrained=True)


### Freezing Layers

In [ ]:
for param in vgg16.features.parameters():
    param.require_grad = False

### Modification In layers

In [ ]:
num_features = vgg16.classifier[6].in_features
classifier = list(vgg16.classifier.children())[:-1]
classifier.extend([nn.Linear(num_features, len(class_names))])
vgg16.classifier = nn.Sequential(*classifier)



In [ ]:
class FocalLoss(torch.nn.Module):
    
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

        
    def forward(self, outputs, labels):
        outputs_sigmoid = torch.sigmoid(outputs)
        BCE_loss = F.binary_cross_entropy(outputs_sigmoid, labels, reduce=False)   
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * torch.Tensor.cpu((1-pt)**self.gamma * BCE_loss).detach().numpy()
        return np.mean(F_loss)

 ### Function & Optimizer

In [ ]:

criterion = FocalLoss([0.4, 0.3, 0.2], 2)
vgg16_optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

### Training

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

vgg16.train()


vgg16_loss = []
vgg16_accuracy = []

# Running Epochs
for epoch in range(Epochs):

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels, paths = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        vgg16_optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vgg16(inputs)               #----> forward pass
        
        # One-Hot encoding labels
        encoder = OneHotEncoder(handle_unknown='ignore')
        labels_in_numpy = torch.Tensor.cpu(labels).detach().numpy().reshape(-1, 1)
        categories = np.array([[0], [1], [2]])
        encoder.fit(categories)
        binary_labels = encoder.transform(labels_in_numpy).toarray()
        
        outputs_numpy = torch.Tensor.cpu(outputs).detach().numpy()
        outputs = torch.from_numpy(binary_labels).to(device)
        labels_numpy = torch.Tensor.cpu(torch.from_numpy(binary_labels)).detach().numpy()
         
        loss = criterion(outputs, torch.from_numpy(binary_labels).to(device))   #----> compute loss
        loss = torch.from_numpy(np.asarray(loss)).to(device)
        loss = Variable(loss, requires_grad = True)
        loss.backward()
        vgg16_optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss))
        
    vgg16_loss.append(loss.data)
        
    torch.save(vgg16.state_dict(), 'vgg16_focal_loss.pth')
    
    correct = 0
    total = 0

    covid_Matrix = np.zeros((2,2))
    pneu_Matrix = np.zeros((2,2))
    normal_Matrix = np.zeros((2,2))
    
    with torch.no_grad():
        for data in validloader:
            images, labels, paths = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            
            # Applying sigmoid
            outputs = 1 / (1 + np.exp(-torch.Tensor.cpu(outputs).detach().numpy()))
            
            # Converting to 1 if greater than threshold of 0.5
            # if no value is greater than the threshold then defaulting to convert the max value to 1
            outputs[:] = np.where(outputs == outputs.max(axis=1).reshape(-1, 1), 1, outputs)
            outputs = np.where(outputs >= 0.5, 1, 0)
            
            # Converting output back to tensor 
            outputs = torch.from_numpy(outputs).to(device)
            
            # One-Hot encoding labels
            encoder = OneHotEncoder(handle_unknown='ignore')
            labels_in_numpy = torch.Tensor.cpu(labels).detach().numpy().reshape(-1, 1)
            categories = np.array([[0], [1], [2]])
            encoder.fit(categories)
            binary_labels = encoder.transform(labels_in_numpy).toarray()
            
            # Setting pneumonia to 1 if covid-19 is seen
            for row in range(binary_labels.shape[0]):
                if binary_labels[row][0] == 1:
                    binary_labels[row][2] = 1
                    
            binary_labels = torch.from_numpy(binary_labels).to(device)
               
            # Converting to numpy arrays for comparison
            outputs = torch.Tensor.cpu(outputs).detach().numpy()
            binary_labels = torch.Tensor.cpu(binary_labels).detach().numpy()
            
            
            total += labels.size(0)
            correct += np.all(outputs==binary_labels, axis=1).sum()
            
            # Computing confusion matrices
            mat = multilabel_confusion_matrix(binary_labels, outputs)
            
            ## Making Confusion Matrix

            normal_Matrix[0][0] += mat[2][0][0]
            normal_Matrix[0][1] += mat[2][0][1]
            normal_Matrix[1][0] += mat[2][1][0]
            normal_Matrix[1][1] += mat[2][1][1]
            covid_Matrix[0][0]  += mat[0][0][0]
            covid_Matrix[0][1]  += mat[0][0][1]
            covid_Matrix[1][0] += mat[0][1][0]
            covid_Matrix[1][1] += mat[0][1][1]
            pneu_Matrix[0][0]  += mat[1][0][0]
            pneu_Matrix[0][1] += mat[1][0][1]
            pneu_Matrix[1][0] += mat[1][1][0]
            pneu_Matrix[1][1] += mat[1][1][1]
            

    
    
    vgg16_accuracy.append((100 * correct / total))

### Scores and Confusion Matrix

In [ ]:
    
print('********* VGG-16 Focal Loss Confusion Matrices*********')
Accuracy_Scores(outputs,binary_labels,correct,total)
 
print('\n\n Normal ')
print(pd.DataFrame(normal_Matrix, A_labels, P_labels))
print('\n\n Covid-19 ')
print(pd.DataFrame(covid_Matrix, A_labels, P_labels))
print('\n\n Pneumonia ')
print(pd.DataFrame(pneu_Matrix, A_labels, P_labels))


### Accuracy and Loss Curve

In [ ]:
print('********* VGG-16 Focal Loss*********')

Graph(Epochs,vgg16_accuracy,vgg16_loss)




### Testing

In [ ]:
# Decalring a Dataframe
testing_df = pd.DataFrame()

with torch.no_grad():
    for data in testloader:
        images, labels, paths = data
        images, labels = images.to(device), labels.to(device)
        
        outputs = vgg16(images)
        
        # Applying sigmoid
        outputs = 1 / (1 + np.exp(-torch.Tensor.cpu(outputs).detach().numpy()))
            
        # Converting to 1 if greater than threshold of 0.5
        # if no value is greater than the threshold then defaulting to convert the max value to 1
        outputs[:] = np.where(outputs == outputs.max(axis=1).reshape(-1, 1), 1, outputs)
        outputs = np.where(outputs >= 0.5, 1, 0)
      
        Imagepaths = pd.DataFrame(paths)[0].str.split("/", expand = True).iloc[:,-1]
        batch_df = pd.concat([Imagepaths, pd.DataFrame(outputs)], axis=1, join='inner')
        columns_titles = [9, 0, 2, 1]
        batch_df = batch_df.reindex(columns=columns_titles)
        batch_df.columns = [''] * len(batch_df.columns)
        batch_df.index = [''] * len(batch_df.index)
        
        testing_df = testing_df.append(batch_df)

testing_df.reset_index(drop=True)

testing_df.to_csv('MSDS19033_results3.csv', index=False, header=False)


# ResNet-18

In [ ]:
resnet18 = models.resnet18(pretrained=True)

In [ ]:
for param in resnet18.parameters():
    param.require_grad = False

In [ ]:
resnet18.fc = nn.Sequential(OrderedDict([('fc1', nn.Linear(512, len(class_names)))]))


In [ ]:

criterion = FocalLoss([0.4, 0.3, 0.2], 2)
resnet18_optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)

## Training

In [ ]:
# Setting to either GPU or CPU based on availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet18.to(device)

resnet18.train()


resnet18_cross_entropy = []
resnet18_valid_accuracy = []

# Running Epochs
for epoch in range(Epochs):

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels, paths = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        resnet18_optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet18(inputs)               #----> forward pass
                
        # One-Hot encoding labels
        encoder = OneHotEncoder(handle_unknown='ignore')
        labels_in_numpy = torch.Tensor.cpu(labels).detach().numpy().reshape(-1, 1)
        categories = np.array([[0], [1], [2]])
        encoder.fit(categories)
        binary_labels = encoder.transform(labels_in_numpy).toarray()
        
        outputs_numpy = torch.Tensor.cpu(outputs).detach().numpy()
        outputs = torch.from_numpy(binary_labels).to(device)
        labels_numpy = torch.Tensor.cpu(torch.from_numpy(binary_labels)).detach().numpy()
         
        loss = criterion(outputs, torch.from_numpy(binary_labels).to(device))   #----> compute loss
        loss = torch.from_numpy(np.asarray(loss)).to(device)
        loss = Variable(loss, requires_grad = True)
        
        loss.backward()                     #----> backward pass
        resnet18_optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))
        
    resnet18_cross_entropy.append(loss.data)
        
    torch.save(resnet18.state_dict(), 'res18_focal_loss.pth')
    correct = 0
    total = 0

    covid_Matrix = np.zeros((2,2))
    pneu_Matrix = np.zeros((2,2))
    normal_Matrix = np.zeros((2,2))
    
    with torch.no_grad():
        for data in validloader:
            images, labels, paths = data
            images, labels = images.to(device), labels.to(device)
            outputs = resnet18(images)
            
            # Applying sigmoid
            outputs = 1 / (1 + np.exp(-torch.Tensor.cpu(outputs).detach().numpy()))
            
            # Converting to 1 if greater than threshold of 0.5
            # if no value is greater than the threshold then defaulting to convert the max value to 1
            outputs[:] = np.where(outputs == outputs.max(axis=1).reshape(-1, 1), 1, outputs)
            outputs = np.where(outputs >= 0.5, 1, 0)
            
            # Converting output back to tensor 
            outputs = torch.from_numpy(outputs).to(device)
            
            # One-Hot encoding labels
            encoder = OneHotEncoder(handle_unknown='ignore')
            labels_in_numpy = torch.Tensor.cpu(labels).detach().numpy().reshape(-1, 1)
            categories = np.array([[0], [1], [2]])
            encoder.fit(categories)
            binary_labels = encoder.transform(labels_in_numpy).toarray()
            
            # Setting pneumonia to 1 if covid-19 is seen
            for row in range(binary_labels.shape[0]):
                if binary_labels[row][0] == 1:
                    binary_labels[row][2] = 1
                    
            binary_labels = torch.from_numpy(binary_labels).to(device)
               
            # Converting to numpy arrays for comparison
            outputs = torch.Tensor.cpu(outputs).detach().numpy()
            binary_labels = torch.Tensor.cpu(binary_labels).detach().numpy()
            total += labels.size(0)
            correct += np.all(outputs==binary_labels, axis=1).sum()
            
            # Computing confusion matrices
            mat = multilabel_confusion_matrix(binary_labels, outputs)
            
            ## Making Confusion Matrix

            normal_Matrix[0][0] += mat[2][0][0]
            normal_Matrix[0][1] += mat[2][0][1]
            normal_Matrix[1][0] += mat[2][1][0]
            normal_Matrix[1][1] += mat[2][1][1]
            covid_Matrix[0][0]  += mat[0][0][0]
            covid_Matrix[0][1]  += mat[0][0][1]
            covid_Matrix[1][0] += mat[0][1][0]
            covid_Matrix[1][1] += mat[0][1][1]
            pneu_Matrix[0][0]  += mat[1][0][0]
            pneu_Matrix[0][1] += mat[1][0][1]
            pneu_Matrix[1][0] += mat[1][1][0]
            pneu_Matrix[1][1] += mat[1][1][1]
            

    
    
    resnet18_accuracy.append((100 * correct / total))

### Scores and Confusion Matrix

In [ ]:
print('********* ResNet18 Confusion Matrices*********')
Accuracy_Scores(outputs,binary_labels,correct,total)

print('\n\n Normal ')
print(pd.DataFrame(normal_Matrix, A_labels, P_labels))
print('\n\n Covid-19 ')
print(pd.DataFrame(covid_Matrix, A_labels, P_labels))
print('\n\n Pneumonia ')
print(pd.DataFrame(pneu_Matrix, A_labels, P_labels))


### Accuracy and Loss Curve

In [ ]:
print('********* ResNet18 *********')
Graph(Epochs,resnet18_accuracy,resnet18_loss)


### Testing

In [ ]:
# Declaring a Dataframe
testing_df = pd.DataFrame()

with torch.no_grad():
    for data in testloader:
        images, labels, paths = data
        images, labels = images.to(device), labels.to(device)
        
        outputs = resnet18(images)
        
        # Applying sigmoid
        outputs = 1 / (1 + np.exp(-torch.Tensor.cpu(outputs).detach().numpy()))
            
        # Converting to 1 if greater than threshold of 0.5
        # if no value is greater than the threshold then defaulting to convert the max value to 1
        outputs[:] = np.where(outputs == outputs.max(axis=1).reshape(-1, 1), 1, outputs)
        outputs = np.where(outputs >= 0.5, 1, 0)
      
        Imagepaths = pd.DataFrame(paths)[0].str.split("/", expand = True).iloc[:,-1]
        batch_df = pd.concat([Imagepaths, pd.DataFrame(outputs)], axis=1, join='inner')
        columns_titles = [9, 0, 2, 1]
        batch_df = batch_df.reindex(columns=columns_titles)
        batch_df.columns = [''] * len(batch_df.columns)
        batch_df.index = [''] * len(batch_df.index)
        
        testing_df = testing_df.append(batch_df)

testing_df.reset_index(drop=True)

testing_df.to_csv('MSDS19033_results4.csv', index=False, header=False)
